<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/app_correcciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title INICIALIZACIÓN
# Verificar e instalar el paquete 'boto3'
try:
    import boto3
except ImportError:
    print("El paquete 'boto3' no está instalado. Procediendo con la instalación...")
    !pip install boto3
    import boto3
    print("El paquete 'boto3' ha sido instalado correctamente.")
from google.cloud import storage
import os
from google.colab import auth
from google.colab import files


# Autenticación para Google Cloud Storage
auth.authenticate_user()

In [ ]:
# @title s3_to_gcs_transfer_file()
def s3_to_gcs_transfer_file(config):
    """
    Transfiere uno o varios archivos desde un bucket S3 a un bucket de Google Cloud Storage.

    :param config: Diccionario con las claves:
        - s3_bucket_name: Nombre del bucket en S3.
        - s3_folder_name: Carpeta dentro del bucket en S3.
        - s3_file_name: Lista de nombres de archivos en el bucket S3.
        - gcs_bucket_name: Nombre del bucket en Google Cloud Storage.
        - gcs_folder_name: Carpeta dentro del bucket en Google Cloud Storage.
    """
    s3_bucket_name = config['s3_bucket_name']
    s3_folder_name = config['s3_folder_name']
    s3_file_names = config['s3_file_name']
    gcs_bucket_name = config['gcs_bucket_name']
    gcs_folder_name = config['gcs_folder_name']

    # Descargar y subir cada archivo
    s3 = boto3.client('s3')
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)

    # Crear carpeta en GCS si no existe
    if gcs_folder_name:
        folder_blob = bucket.blob(f"{gcs_folder_name}/")
        if not folder_blob.exists():
            folder_blob.upload_from_string('')

    for file_name in s3_file_names:
        s3_key = f"{s3_folder_name}/{file_name}" if s3_folder_name else file_name
        gcs_key = f"{gcs_folder_name}/{file_name}" if gcs_folder_name else file_name
        temp_file_path = f"/tmp/{file_name}"  # Ruta temporal en el sistema local

        # Descargar el archivo desde S3
        try:
            print(f"Descargando {s3_key} desde el bucket {s3_bucket_name}...")
            s3.download_file(s3_bucket_name, s3_key, temp_file_path)
            print(f"Archivo descargado en {temp_file_path}")
        except Exception as e:
            print(f"Error al descargar el archivo {file_name} desde S3: {e}")
            continue

        # Subir el archivo a Google Cloud Storage
        try:
            print(f"Subiendo {gcs_key} al bucket {gcs_bucket_name}...")
            blob = bucket.blob(gcs_key)
            blob.upload_from_filename(temp_file_path)
            print(f"Archivo subido a {gcs_bucket_name}/{gcs_key}")
        except Exception as e:
            print(f"Error al subir el archivo {file_name} a Google Cloud Storage: {e}")
        finally:
            # Limpiar el archivo temporal
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)

In [ ]:
# @title COPIADO DE S3 A GCS
from google.colab import userdata
AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')

s3_bucket_name = "plataforma-alol"          # @param {type:"string"}
s3_folder_name = "Contenidos/videos/correcciones"  # @param {type:"string"}
s3_file_name = ["C3362490.mp4"]  # @param {type:"string"}
gcs_bucket_name = "mocadi"       # @param {type:"string"}
gcs_folder_name = "test"    # @param {type:"string"}

if __name__ == "__main__":
    # Configura los valores según tu entorno
    config = {
        's3_bucket_name': s3_bucket_name,
        's3_folder_name': s3_folder_name,
        's3_file_name': s3_file_name,
        'gcs_bucket_name': gcs_bucket_name,
        'gcs_folder_name': gcs_folder_name
    }

    # Establecer credenciales de AWS mediante variables de entorno
    os.environ['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
    os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY
    os.environ['AWS_DEFAULT_REGION'] = 'eu-west-1'

    # Ejecuta la transferencia
    s3_to_gcs_transfer_file(config)


Descargando Contenidos/videos/correcciones/C3362490.mp4 desde el bucket plataforma-alol...
Archivo descargado en /tmp/C3362490.mp4
Subiendo test/C3362490.mp4 al bucket mocadi...
Archivo subido a mocadi/test/C3362490.mp4
